In [ ]:
import os
import torch
import numpy as np

print(os.sys.path)
os.sys.path.append("/home/matthias/DIKU Project Protein Folding/protein-folding-gans/src/openprotein/pnerf")

# CODE FILES HERE
from model_params import get_model_data_gan
from models.gan import Gan, Generator, Discriminator
from solver import Solver
from directories import Directories
from dataloader import DataLoader
from plots import plot_losses, plot_z_samples
from sampling import gan_sampling
from openprotein.preprocessing import process_raw_data

# SETTINGS HERE
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # to see the CUDA stack
%matplotlib inline
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
# supress cluttering warnings in solutions
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# setting device on GPU if available, else CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

In [ ]:
dataset_arg = "proteins"
data = get_model_data_gan(dataset_arg)

In [ ]:
training_file = "openprotein/data/preprocessed/testing.txt.hdf5"
validation_file = "openprotein/data/preprocessed/testing.txt.hdf5"
process_raw_data(False, force_pre_processing_overwrite=False)

In [ ]:
directories = Directories("gan", dataset_arg.lower(), data["z_dim"], make_dirs=True)
data_loader = DataLoader(directories, data["batch_size"], dataset_arg.lower(),
                         training_file=training_file, validation_file=validation_file,
                         residue_fragments=data["residue_fragments"], atom="calpha")
gan = Gan(data_loader.input_dim, data["z_dim"])
generator = Generator(data["z_dim"], data_loader.input_dim, data_loader.img_dims)
discriminator = Discriminator(data_loader.input_dim, 1)
solver = Solver(gan, generator, discriminator, data["epochs"], data_loader, data["optimizer_G"],
                data["optimizer_D"], data["optim_config_G"], data["optim_config_D"], save_model_state=False)
solver.main()

In [ ]:
# Insert name of model here if want to load a model
#solver = torch.load(solver.save_model_dir + "/model_GAN_mnist_z=100.pt")
#solver.model.eval()
#solver.data_loader.directories.make_dirs = False

In [ ]:
# Plotting g and d losses for all epochs
plot_losses(solver, solver.train_loss_history["g_loss"], solver.train_loss_history["d_loss"])

In [ ]:
samples = gan_sampling(generator, solver.model.z_dim, 100)
plot_z_samples(samples, solver.data_loader.directories.make_dirs,
               solver.data_loader.directories.result_dir, solver.data_loader.dataset)